In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Set a threshold for binary conversion
    ret, thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area (assuming it's the hand)
    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)

        # Draw the largest contour
        cv2.drawContours(frame, [max_contour], -1, (0, 255, 0), 2)

        # Get convex hull
        hull = cv2.convexHull(max_contour, returnPoints=False)

        # Get defects in convex hull
        defects = cv2.convexityDefects(max_contour, hull)

        # Count fingers
        finger_count = 0

        if defects is not None:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(max_contour[s][0])
                end = tuple(max_contour[e][0])
                far = tuple(max_contour[f][0])

                a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)

                angle = (np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / np.pi

                if angle <= 90:
                    finger_count += 1
                    cv2.circle(frame, far, 5, [0, 0, 255], -1)

        # Display the count of fingers
        cv2.putText(frame, str(finger_count + 1), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Hand Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
